# Sequences selected by divergent exhibit large conventional genetic distance

We test the expectation that the minimum pairwise genetic distance sequences between sequences selected by divergent max is greater than the corresponding minimum between a random selection of the same number of sequences.

The data was from alignments of 106 genes from exactly 31 diverse mammal species. The matrix of all pairwise genetic distances were computed between the aligned sequences using the Paralinear distance of Lake. If the JSD measure reflects genetic distance, we expect the mean of the pairwise distances between the species identified by `divergent max` to be larger than the same number of species selected randomly. We set a minimal set size of 5 and maximal set size of 10.

In [ ]:
import plotly.express as px
import project_path
from cogent3 import load_table
from plotly.subplots import make_subplots

write_pdf = project_path.pdf_writer()

# style settings
cols = {
    "stdev": dict(color="blue"),
    "cov": dict(color="red"),
    "JSD": dict(color="cyan"),
}
opacity = 0.6
marker = dict(size=18, opacity=opacity)
d_jsd = "δ<sub>JSD</sub>"
stat_names = dict(stdev=f"std({d_jsd})", cov=f"cov({d_jsd})", JSD=r"JSD(<b>F</b>)")
tickfont = dict(size=20)
titlefont = dict(size=26)
legend = dict(title=dict(text=""), font=dict(size=28), tracegroupgap=10)


table = load_table(project_path.RESULT_DIR / "jsd_v_dist-stats.tsv")
# eliminating rows where min_size == max_size
table = table.filtered(lambda x: len(set(x)) > 1, columns=["min_size", "max_size"])
other = load_table(project_path.RESULT_DIR / "jsd_v_dist-nmost.tsv")
table = table.appended(None, other)
table

## The value of $k$ impacts on the dispersal of the selected sequences

The performance of divergent `max` in recovering representative sequences is a function of $k$ and the selected statistic.

In [ ]:
fig_stat = px.scatter(
    table,
    x="k",
    y="(p-value<0.05)%",
    color="stat",
    trendline="lowess",
    labels={
        "k": "<i>k</i>",
        "(p-value<0.05)%": "Significant %",
        "fixed_size": "Fixed Set Size",
    },
)

## The distribution of the number of selected sequences

Both $k$ and the statistic impact on the number of sequences selected by divergent `max`.

In [ ]:
import statsmodels.api as sm
from cogent3 import load_table, make_table
from plotly.graph_objects import Figure, Scatter


def make_trendline(x, y, name):
    lowess = sm.nonparametric.lowess(x, y)
    lowess_x = lowess[:, 0]
    lowess_y = lowess[:, 1]
    return Scatter(
        x=lowess_x,
        y=lowess_y,
        mode="lines",
        name=name,
        line=cols[name],
        showlegend=False,
        opacity=0.5,
    )


def make_trace(table, name, fig):
    st = Scatter(
        x=table.columns["k"],
        y=table.columns["mean_size"],
        mode="markers",
        name=name,
        error_y=dict(
            type="data",
            array=table.columns["error_above"].tolist(),
            arrayminus=table.columns["error_below"].tolist(),
        ),
        marker=cols[name] | marker,
    )
    fig.add_trace(st)
    fig.add_trace(
        make_trendline(table.columns["mean_size"], table.columns["k"], name),
    )


def make_plot(table):
    # this has to be done manually because plotly express has a
    # bug with error bars and multiple traces
    stdev = table.filtered(lambda x: x == "stdev", columns="stat")
    cov = table.filtered(lambda x: x == "cov", columns="stat")
    fig = Figure()
    make_trace(stdev, "stdev", fig)
    make_trace(cov, "cov", fig)
    return fig


def calc_upper_lower_bounds(x, y, lower, upper):
    assert lower <= x <= upper, f"invalid {x=}, {lower=}, {upper=}"
    lo = x - y
    hi = x + y
    if lo >= lower and hi <= upper:
        return y, y
    if lo <= lower and hi >= upper:
        return x - lower, upper - x
    if lo < lower:
        diff = x - lower
        return x - lower, y - diff
    diff = upper - x
    return (y - diff), upper - x


def group_by(table, one_stat):
    columns = ["k", "min_size", "max_size", "stat"]
    distinct = table.distinct_values(columns)

    results = []
    for group in distinct:
        _, lower, upper, _ = group
        subtable = table.filtered(lambda x: tuple(x) == group, columns=columns)
        sizes = subtable.columns["size"]
        mean, std = sizes.mean(), sizes.std(ddof=1)
        lo, hi = calc_upper_lower_bounds(mean, std, lower, upper)
        results.append(list(group) + [mean, std, lo, hi, sizes.shape[0]])

    table = make_table(
        header=columns + ["mean_size", "std_size", "error_below", "error_above", "n"],
        rows=results,
    )
    table = table.sorted(columns=("stat", "k"))
    if one_stat:
        table = table.filtered(lambda x: x == "stdev", columns="stat")
    return table


table = load_table(
    project_path.RESULT_DIR / "jsd_v_dist-sizes.tsv",
).filtered(lambda x: x == 5, columns="min_size")
table = group_by(table, one_stat=False)
fig_size = make_plot(table)

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
for trace in fig_stat.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig_size.data:
    trace.showlegend = False
    fig.add_trace(trace, row=2, col=1)

fig.update_yaxes(title_text="<b>Significant %</b>", row=1, col=1, title_font=titlefont)
fig.update_yaxes(
    title_text="<b>Mean Number Selected</b>", row=2, col=1, title_font=titlefont
)
fig.update_xaxes(title_text="<b><i>k</i></b>", row=2, col=1, title_font=titlefont)
fig.update_layout(legend=legend, width=1600, height=1200)
fig.for_each_trace(lambda t: t.update(name=stat_names.get(t.name, t.name)))
fig.update_traces(
    selector=dict(name=stat_names["cov"]),
    marker=cols["cov"] | marker,
    line=cols["cov"],
)
fig.update_traces(
    selector=dict(name=stat_names["stdev"]),
    marker=cols["stdev"] | marker,
    line=cols["stdev"],
)
fig.update_traces(
    selector=dict(name=stat_names["JSD"]),
    marker=cols["JSD"] | marker,
    line=cols["JSD"],
)
fig.update_traces(opacity=opacity, selector=dict(mode="lines"))
fig.update_traces(marker=marker, selector=dict(mode="markers"))
fig.update_yaxes(tickfont=tickfont, row=1, col=1)
fig.update_yaxes(tickfont=tickfont, row=2, col=1)
fig.update_xaxes(tickfont=tickfont, row=2, col=1)

x, y = 0.2, 1.05
fig.add_annotation(
    xref="x1",
    yref="y1",
    x=x * 10,
    y=y * 100,
    text="<b>(a)</b>",
    showarrow=False,
    row=1,
    col=1,
    font=titlefont,
)

fig.add_annotation(
    xref="paper",
    yref="paper",
    x=x * 10,
    y=y * 30,
    text="<b>(b)</b>",
    showarrow=False,
    row=2,
    col=1,
    font=titlefont,
)
outpath = project_path.FIG_DIR / "jsd_v_dist.pdf"
fig.show()
# write_pdf(fig, outpath)